Import revelant packages

In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold

Read in the data set, split into data (`x`) and labels (`y`) then scale the data 

In [ ]:
data = pd.read_csv("Mixcancer.csv")
y = np.asarray(data.pop('Diagnosis'))
x = np.asarray(data)

x = preprocessing.MinMaxScaler().fit_transform(x)

Split data and labels into test & train sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=0)


Sigmoid activation & derivative

In [ ]:
def sigmoid(v):
    return 1/(1+np.exp(-v))

def sigmoid_derivative(v):
    return sigmoid(v)*(1-sigmoid(v))

Relu activation & derivative

In [ ]:
def LReLU(x):
    return np.where(x > 0, x, x * 0.01)

def dLReLU(x):
    return np.where(x > 0, 1, 0.01)

Cross entropy & derivative

In [ ]:
def crossEntrop(o,y):
    return (-y*(np.log(o)) - (1-y)* np.log(1-o))

def crossEntrDeriv(o,y):
  return -(y/o - (1-y)/(1-o))

Define layer sizes

In [ ]:
layer_1_size = 5
layer_2_size = 1

Initialise variables for training

In [ ]:
epochs = 5000
lr = 0.005
batch_size = 128
kf = KFold(n_splits=10, shuffle=True)

Train the model

In [ ]:
n = len(x_train[1])
train_E = []
test_E = []
train_Acc = []
test_Acc = []
f_measures = []
fold = 0
for train_index, test_index in kf.split(x):
  w1 = np.random.uniform(-1,1,[x_train.shape[1],layer_1_size])
  w2 = np.random.uniform(-1,1,[layer_1_size])

  b1 = np.random.uniform(-1,1,[layer_1_size])
  b2 = np.random.uniform(-1,1,[layer_2_size])
  for epoch in range(epochs):
    for i in range(0, x_train.shape[0], batch_size):
      x_batch = x_train[i:i + batch_size,:]
      y_batch = y_train[i:i + batch_size]
      if b1.shape != [layer_1_size,]:
        b1 = b1.reshape([layer_1_size,])
      if w2.shape != [layer_1_size,]:
        w2 = w2.reshape([layer_1_size,])
      in1 = x_batch@w1+b1
      out1 = LReLU(in1)
      in2 = out1@w2+b2
      out2 = sigmoid(in2)

      dEdO2 = crossEntrDeriv(out2,y_batch)
      dO2dIn2 = sigmoid_derivative(in2)
      dO1dIn1 = dLReLU(in1)
      
      dEdW2 = 1/n * out1.T@(dEdO2*dO2dIn2)
      dEdB2 = 1/n * np.ones([1,x_batch.shape[0]])@(dEdO2*dO2dIn2)
      
      w2 = w2.reshape(1,w2.shape[0])

      dEdW1 = 1/n * x_batch.T@((np.reshape(dEdO2*dO2dIn2,[len(x_batch),1])@w2)*dO1dIn1)
      dEdB1 = 1/n * np.ones([1,x_batch.shape[0]])@((np.reshape(dEdO2*dO2dIn2,[len(x_batch),1])@w2)*dO1dIn1)

      w1 -= lr*dEdW1
      w2 -= lr*dEdW2
      b1 -= lr*dEdB1[0,:]
      b2 -= lr*dEdB2
    if epoch % 100 == 0:
      error_train = crossEntrop(out2,y_batch)
      b1 = b1.reshape([layer_1_size,])
      w2 = w2.reshape([layer_1_size,])
      train_E.append(error_train.mean())
      print('Epoch:\t%s\tTrain Error:\t%s'%(epoch,train_E[-1]))
      prediction_test = np.where(sigmoid(LReLU(x_test@w1+b1)@w2+b2) > 0.5,1,0)
  fold +=1
  f_measure = metrics.f1_score(y_test,prediction_test)
  f_measures.append(f_measure)
  print('Fold:\t%s\tF-Measure:\t%s'%(fold,f_measure))
print('Macro-F-Measure:\t%s'%(np.array(f_measures).mean()))


Epoch:	0	Train Error:	2.2430372520808746
Epoch:	100	Train Error:	0.5287384926687246
Epoch:	200	Train Error:	0.34090211175453494
Epoch:	300	Train Error:	0.23832176449842826
Epoch:	400	Train Error:	0.19113652167837822
Epoch:	500	Train Error:	0.16617569944415697
Epoch:	600	Train Error:	0.1508480682627091
Epoch:	700	Train Error:	0.1403092182695178
Epoch:	800	Train Error:	0.13247151987249003
Epoch:	900	Train Error:	0.12627782098610713
Epoch:	1000	Train Error:	0.12117914304375096
Epoch:	1100	Train Error:	0.11681404807837276
Epoch:	1200	Train Error:	0.11300289633562886
Epoch:	1300	Train Error:	0.10966334878910272
Epoch:	1400	Train Error:	0.10663563234680543
Epoch:	1500	Train Error:	0.10392299476872094
Epoch:	1600	Train Error:	0.10141179430427356
Epoch:	1700	Train Error:	0.09915086609726245
Epoch:	1800	Train Error:	0.09701805458227215
Epoch:	1900	Train Error:	0.0950528129026409
Epoch:	2000	Train Error:	0.09322548775031847
Epoch:	2100	Train Error:	0.09150899459369287
Epoch:	2200	Train Error:	0.